In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs
     


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [69]:

from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [70]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [72]:

from typing import Tuple, List, Optional

In [73]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
     

In [74]:
from langchain_core.runnables import ConfigurableField
     

from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

     

import os

In [75]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass
     

from langchain_community.vectorstores import Neo4jVector

In [76]:
NEO4J_URI="neo4j+s://2c755919.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="RGUjiPrFzwTsjFig0gc743rKroP-PqLYPc0q7fvcSEM"
     

In [77]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [78]:
from langchain_community.graphs import Neo4jGraph

In [79]:
graph = Neo4jGraph()

In [83]:
print(graph.query("MATCH (n) RETURN COUNT(n)"))  # Should return 0

[{'COUNT(n)': 0}]


In [84]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Elizabeth I").load()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [85]:
len(raw_documents)

24

In [86]:
raw_documents[:3]

[Document(metadata={'title': 'Elizabeth I', 'summary': 'Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last and longest reigning monarch of the House of Tudor. Her eventful reign, and its effect on history and culture, gave name to the Elizabethan era.\nElizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents\' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry\'s death in 1547, Elizabeth\'s younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward\'s will was set aside w

In [87]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [88]:
print(type(documents))  # Should be <class 'list'>
print(type(documents[0]))  # Should be <class 'langchain_core.documents.Document'>

<class 'list'>
<class 'langchain_core.documents.base.Document'>


In [92]:
print(documents[0]) 

page_content='Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last and longest reigning monarch of the House of Tudor. Her eventful reign, and its effect on history and culture, gave name to the Elizabethan era.
Elizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry's death in 1547, Elizabeth's younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward's will was set aside within weeks of his death and Mary became queen,

In [93]:
for doc in documents:
    print(type(doc.metadata))  # Should be <class 'dict'>
    for key, value in doc.metadata.items():
        print(f"Key: {key}, Type: {type(value)}")  # Check if any value is a list

<class 'dict'>
Key: title, Type: <class 'str'>
Key: summary, Type: <class 'str'>
Key: source, Type: <class 'str'>
<class 'dict'>
Key: title, Type: <class 'str'>
Key: summary, Type: <class 'str'>
Key: source, Type: <class 'str'>
<class 'dict'>
Key: title, Type: <class 'str'>
Key: summary, Type: <class 'str'>
Key: source, Type: <class 'str'>
<class 'dict'>
Key: title, Type: <class 'str'>
Key: summary, Type: <class 'str'>
Key: source, Type: <class 'str'>
<class 'dict'>
Key: title, Type: <class 'str'>
Key: summary, Type: <class 'str'>
Key: source, Type: <class 'str'>
<class 'dict'>
Key: title, Type: <class 'str'>
Key: summary, Type: <class 'str'>
Key: source, Type: <class 'str'>


In [89]:
from langchain_community.llms import Ollama
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm = Ollama(model="mistral")

In [90]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

# Initialize transformer with the local Ollama model
llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
fixed_documents = [
    Document(
        page_content=str(doc.page_content).strip(),  # Ensure it's a clean string
        metadata={k: tuple(v) if isinstance(v, list) else str(v) for k, v in doc.metadata.items()}
    )
    for doc in documents
]

graph_documents = []
for i, doc in enumerate(fixed_documents):
    try:
        result = llm_transformer.convert_to_graph_documents([doc])  # Process individually
        graph_documents.extend(result)
        print(f"✅ Document {i} processed successfully")
    except Exception as e:
        print(f"❌ Error in Document {i}: {e}")
        print(f"Metadata of failing doc: {doc.metadata}")
        break  # Stop at the first failing document

✅ Document 0 processed successfully
❌ Error in Document 1: unhashable type: 'list'
Metadata of failing doc: {'title': 'Elizabeth I', 'summary': 'Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last and longest reigning monarch of the House of Tudor. Her eventful reign, and its effect on history and culture, gave name to the Elizabethan era.\nElizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents\' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry\'s death in 1547, Elizabeth\'s younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and th

In [110]:
for key, value in documents[1].metadata.items():
    print(f"Key: {key}, Type: {type(value)}, Value: {value}")

Key: title, Type: <class 'str'>, Value: Elizabeth I
Key: summary, Type: <class 'str'>, Value: Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last and longest reigning monarch of the House of Tudor. Her eventful reign, and its effect on history and culture, gave name to the Elizabethan era.
Elizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry's death in 1547, Elizabeth's younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contra

In [120]:
graph_documents = []

for i, doc in enumerate(fixed_documents):
    try:
        result = llm_transformer.convert_to_graph_documents([doc])  # Process one at a time
        graph_documents.extend(result)
        print(f"✅ Document {i} processed successfully")
    except Exception as e:
        print(f"❌ Error in Document {i}: {e}")
        print(f"Metadata of failing doc: {doc.metadata}")
        break  # Stop at first failure

✅ Document 0 processed successfully
✅ Document 1 processed successfully
✅ Document 2 processed successfully
✅ Document 3 processed successfully
✅ Document 4 processed successfully
✅ Document 5 processed successfully


In [121]:
graph_documents

[GraphDocument(nodes=[Node(id='James VI of Scotland', type='Person', properties={}), Node(id='Mary I', type='Person', properties={}), Node(id='William Cecil', type='Person', properties={}), Node(id='Burghley', type='Title', properties={}), Node(id='England', type='Country', properties={}), Node(id='Edward VI', type='Person', properties={}), Node(id='Elizabeth I', type='Person', properties={}), Node(id='Henry VIII', type='Person', properties={}), Node(id='Lady Jane Grey', type='Person', properties={}), Node(id='Ireland', type='Country', properties={}), Node(id='House of Tudor', type='Dynasty', properties={})], relationships=[Relationship(source=Node(id='Elizabeth I', type='Person', properties={}), target=Node(id='England', type='Country', properties={}), type='RULES_ON', properties={}), Relationship(source=Node(id='Elizabeth I', type='Person', properties={}), target=Node(id='Ireland', type='Country', properties={}), type='RULES_ON', properties={}), Relationship(source=Node(id='Elizabeth

In [122]:

graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)
     

In [123]:
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [124]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [125]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [126]:

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [127]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [128]:

from typing import Tuple, List, Optional

In [129]:
from langchain_community.vectorstores import Neo4jVector

In [133]:
from langchain_community.vectorstores import Neo4jVector
from langchain_community.embeddings import OllamaEmbeddings

# Use Mistral embeddings (4096 dimensions)
ollama_embeddings = OllamaEmbeddings(model="mistral")

# Ensure both indexes reference the same node label (Chunk)
vector_index = Neo4jVector.from_existing_graph(
    ollama_embeddings,
    search_type="hybrid",
    node_label="Chunk",  # Must match both indexes
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

print("✅ Neo4j Vector Index Created Successfully with 4096 dimensions")

✅ Neo4j Vector Index Created Successfully with 4096 dimensions


In [134]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [135]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [136]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [137]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)
   

In [144]:
!pip3 install ollama


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [149]:
import json
import ollama

# Define the input question
input_question = "Where was Amelia Earhart born?"

# Create a structured prompt enforcing JSON list output
prompt = f"""
You are an AI that extracts entity names.
Return **only** a JSON list of entity names, nothing else.

### Output Format:
["entity1", "entity2", ...]

Input: "{input_question}"
Output:
"""

# Call Mistral using Ollama
response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])

# Extract and parse response
response_text = response['message']['content'].strip()

try:
    entity_names = json.loads(response_text)  # Convert response to Python list
    print(entity_names)  # Equivalent to `.names`
except json.JSONDecodeError:
    print("Model did not return valid JSON. Raw output:", response_text)

['Amelia Earhart']


In [150]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [162]:
import ollama
import json
from langchain_neo4j.vectorstores.neo4j_vector import remove_lucene_chars
from typing import List, Tuple
from langchain.schema import HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# Function to generate a full-text search query
def generate_full_text_query(input: str) -> str:
    words = [el for el in remove_lucene_chars(input).split() if el]
    full_text_query = " AND ".join([f"{word}~2" for word in words])
    return full_text_query.strip()

# Function to extract entities using Mistral (Ollama)
def extract_entities(question: str):
    stop_words = {"house", "city", "place", "country", "state", "thing", "object", "concept"}  # Common irrelevant words
    prompt = f"""
    You are an AI that extracts entity names from text.
    Return only a JSON list of **proper entity names** (people, organizations, historical events). 
    Do NOT include generic words like 'house', 'place', 'country', 'state'.

    ### Output Format:
    ["entity1", "entity2", ...]

    Input: "{question}"
    Output:
    """

    response = ollama.chat(model="mistral", messages=[{"role": "user", "content": prompt}])
    response_text = response['message']['content'].strip()

    print("🔹 Raw response from Mistral:", response_text)  # Debugging output

    try:
        entities = json.loads(response_text)
        if isinstance(entities, list):
            return [e for e in entities if e.lower() not in stop_words]  # Remove stop words
    except json.JSONDecodeError:
        print("❌ Model did not return valid JSON. Raw output:", response_text)

    return []

# Function to retrieve structured knowledge from Neo4j
def structured_retriever(question: str) -> str:
    print(f"🔍 Structured Search Query: {question}")
    result = ""
    entities = extract_entities(question)  # Extract entities using Mistral

    if not entities:
        print("⚠️ No entities extracted. Check Mistral response format.")
        return ""

    for entity in entities:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node, score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
            } WITH output
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    
    return result

# Unified retriever function (Structured + Unstructured Data)
def retriever(question: str):
    print(f"🔍 Search Query: {question}")
    structured_data = structured_retriever(question)  # Retrieve structured knowledge from Neo4j
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]  # Retrieve from Vector DB
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

# Prompt Template for Condensing Follow-Up Questions
_template = """Given the following conversation and a follow-up question, rephrase the follow-up question to be a standalone question, in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

# Format chat history for input processing
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

# Determines if a follow-up question needs to be converted into a standalone question
_search_query = RunnableBranch(
    # If chat history exists, condense the question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(run_name="HasChatHistoryCheck"),
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | (lambda x: ollama.chat(model="mistral", messages=[{"role": "user", "content": x.to_string()}])['message']['content'])  # Convert to string
        | StrOutputParser(),
    ),
    # Else, use the question directly
    RunnableLambda(lambda x : x["question"]),
)

# Template for generating final answer
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

prompt = ChatPromptTemplate.from_template(template)

# Runnable Chain to Retrieve Data & Generate Answers
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | (lambda x: ollama.chat(model="mistral", messages=[{"role": "user", "content": x.to_string()}])['message']['content'])  # Convert to string before sending
    | StrOutputParser()
)

# Running the pipeline for an initial query
print(chain.invoke({"question": "Which house did Elizabeth I belong to?"}))

# Running the pipeline with follow-up history
print(chain.invoke(
    {
        "question": "When was she born?",
        "chat_history": [("Which house did Elizabeth I belong to?", "House Of Tudor")],
    }
))

🔍 Search Query: Which house did Elizabeth I belong to?
🔍 Structured Search Query: Which house did Elizabeth I belong to?
🔹 Raw response from Mistral: ["Elizabeth I"]


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node) { ... }} {position: line: 3, column: 13, offset: 105} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node, score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output\n            } WITH output\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.

 Elizabeth I belonged to the House of Tudor.
🔍 Search Query:  At what year was Elizabeth I born?
🔍 Structured Search Query:  At what year was Elizabeth I born?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node) { ... }} {position: line: 3, column: 13, offset: 105} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node, score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output\n            } WITH output\n            RETURN output LIMIT 50\n            "


🔹 Raw response from Mistral: ["Elizabeth I"]


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

 Elizabeth I was born on 7 September 1533.
